###  Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [52]:
import requests

In [53]:
from bs4 import BeautifulSoup

In [54]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'
params = {
    'search_query': 'python'
}

In [55]:
req= requests.get(URL,params)
req

<Response [200]>

In [56]:
req.text

'<!DOCTYPE html>\n<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">\n<head >\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover">\n  <title>Все публикации подряд / Хабр</title>\n  <style>\n    /* cyrillic-ext */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format(\'woff2\');\n      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;\n    }\n\n    /* cyrillic */\n    @font-face {\n      font-family: \'Fira Sans\';\n      font-style: normal;\n      font-weight: 500;\n      font-display: swap;\n      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format(\'woff2\');\n      unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;\n    }\n\n 

In [57]:
# soup = BeautifulSoup(req.text, 'html.parser')
soup = BeautifulSoup(req.text)
soup                                 

<!DOCTYPE html>
<html data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D" lang="ru">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width,initial-scale=1.0,viewport-fit=cover" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<style>
    /* cyrillic-ext */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveSxf6TF0.woff2) format('woff2');
      unicode-range: U+0460-052F, U+1C80-1C88, U+20B4, U+2DE0-2DFF, U+A640-A69F, U+FE2E-FE2F;
    }

    /* cyrillic */
    @font-face {
      font-family: 'Fira Sans';
      font-style: normal;
      font-weight: 500;
      font-display: swap;
      src: url(https://fonts.gstatic.com/s/firasans/v11/va9B4kDNxMZdWfMOD5VnZKveQhf6TF0.woff2) format('woff2');
      unicode-range: U+0400-045F, U+0490-0491, U+04B0-04B1, U+2116;
    }

    /* latin-ext */
    @font-face {
      f

In [58]:
#posts = soup.find_all('article', class_='post')
#posts
news_blocks = soup.find_all('article', class_='post')
#len(news_blocks)
news_blocks

[]

In [59]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('div', class_='post__body'), news_blocks))
articles_intro

[]

In [60]:
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
a_list

[]

In [61]:
for post in news_blocks:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

In [62]:
# очень часто сайты могут ограничивать частые запросы к себе, 
# поэтому нужно задерживать исполнение
import time
time.sleep(0.2)

In [63]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
#KEYWORDS = ['python', 'парсинг']
#URL = 'https://habr.com/ru/all/'
#params = {
#    'search_query': 'python'
#}

def get_all_links(URL, KEYWORDS, pages):
    all_refs = []
    params = {
        'search_query': KEYWORDS
    }
    for i in range(1, pages):
        params['page'] = i
        res = requests.get(URL, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('article', class_='post')
        articles_intro = list(map(lambda x: x.find('div', class_='post__body'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        all_refs = a_list
    return all_refs

all_links = get_all_links(URL, 'python', 5)
all_links[]

SyntaxError: invalid syntax (<ipython-input-63-bca51aef34a1>, line 25)

In [64]:
import pandas as pd

In [65]:
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('span', class_='post__time'):
        date = soup.find('span', class_='post__time').text
    print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='post__title')
    else: 
        title = soup.find('h1', class_='post__title')
    print(title)
    if soup.find('h2', class_='post__title'):
        title_element = soup.find('a',class_='post__title_link').text
    print(title_element)
    

In [34]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('span', class_='post__time'):
        date = soup.find('span', class_='post__time').text
    #print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='post__title')
    else: 
        title = soup.find('h1', class_='post__title')
    #print(title)
    if soup.find('h2', class_='post__title'):
        title_element = soup.find('a',class_='post__title_link').text
    #print(title_element)
    row = {'date': date, 'title': title, 'link': link}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news.reset_index()

,index


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

### Задание 2

#### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

#### Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [35]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
EMAIL_2 = {'emailAddresses':['xxx@x.ru']}
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

HEADERS = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}
email_check = requests.post(URL, json=EMAIL_2, headers=HEADERS)
email_check.text

#https://www.avast.com/c-identity-theft

'{"breaches":{"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims\' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised o

In [36]:
soup = BeautifulSoup((email_check).text, 'html.parser')
soup  

#https://www.avast.com/api/v1/tooltip?locale=en-us

{"breaches":{"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a 

In [37]:
news_blocks_1 = soup.find_all('article', class_='post')
#len(news_blocks)
news_blocks_1

[]

In [40]:
email_check
print(email_check.text)

{"breaches":{"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised on a 

In [41]:
print(email_check.content)

b'{"breaches":{"3":{"breachId":3,"site":"adobe.com","recordsCount":152046506,"description":"In October of 2013, criminals penetrated Adobe\'s corporate network and the stole source code for several of its software products. The affected products included Adobe\'s ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.","publishDate":"2016-10-21T00:00:00Z","statistics":{"usernames":0,"passwords":129430596,"emails":152046506}},"12":{"breachId":12,"site":"vk.com","recordsCount":110121799,"description":"Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims\' e-mail addresses, passwords, dates of birth, phone numbers and location details. The credential set was advertised 

In [46]:
email_check.json()

{'breaches': {'3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 129430596,
    'emails': 152046506}},
  '12': {'breachId': 12,
   'site': 'vk.com',
   'recordsCount': 110121799,
   'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers 

In [50]:
email_check.json()['breaches']

{'3': {'breachId': 3,
  'site': 'adobe.com',
  'recordsCount': 152046506,
  'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products included Adobe's ColdFusion web application platform as well as the Acrobat suite of products. Adobe asserts that criminals also accessed nearly three-million customer credit card records and stole login data for an undisclosed number of Adobe user accounts.",
  'publishDate': '2016-10-21T00:00:00Z',
  'statistics': {'usernames': 0, 'passwords': 129430596, 'emails': 152046506}},
 '12': {'breachId': 12,
  'site': 'vk.com',
  'recordsCount': 110121799,
  'description': "Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million clear-text passwords were compromised in the breach. Breached credential sets included victims' e-mail addresses, passwords, dates of birth, phone numbers and location details. The cred

In [51]:
pd.DataFrame(email_check.json()['breaches'])

,3,12,15,2961,3164,3176
breachId,3,12,15,2961,3164,3176
site,adobe.com,vk.com,imesh.com,cdprojektred.com,cfire.mail.ru,parapa.mail.ru
recordsCount,152046506,110121799,50913222,1871459,12880232,5029003
description,"In October of 2013, criminals penetrated Adobe...",Popular Russian social networking platform VKo...,"In June 2016, a cache of over 51 million user ...","In March 2016, CDProjektRed.com.com's forum da...","In July and August of 2016, two criminals carr...","In July and August 2016, two criminals execute..."
publishDate,2016-10-21T00:00:00Z,2016-10-29T00:00:00Z,2016-10-23T00:00:00Z,2017-01-31T00:00:00Z,2017-02-14T00:00:00Z,2017-02-14T00:00:00Z
statistics,"{'usernames': 0, 'passwords': 129430596, 'emai...","{'usernames': 0, 'passwords': 107425364, 'emai...","{'usernames': 50909015, 'passwords': 50913222,...","{'usernames': 1871410, 'passwords': 1871457, '...","{'usernames': 12880232, 'passwords': 12880222,...","{'usernames': 5029000, 'passwords': 5029003, '..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>